In [1]:
from owlready2 import *
#import types
from conllu import parse_incr, parse
import numpy as np
import pandas as pd

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


Carregando ontologias

Carregando Listas de entidedes

In [2]:
#Completo
#text_treino = np.load('../../Corpora/PetroNER-LinkedEntity/PetroNER - sentences.npy')
entities_PetroNER = np.load('../../Corpora/PetroNER-LinkedEntity/PetroNER - entities.npy')
classes_PetroNER = np.load('../../Corpora/PetroNER-LinkedEntity/PetroNER - classes.npy')
URI_PetroNER = np.load('../../Corpora/PetroNER-LinkedEntity/PetroNER - URI.npy', allow_pickle=True)

#Treino
#text_treino = np.load('../../Corpora/PetroNER-LinkedEntity/treino - sentences.npy')
entities_treino = np.load('../../Corpora/PetroNER-LinkedEntity/treino - entities.npy')
classes_treino = np.load('../../Corpora/PetroNER-LinkedEntity/treino - classes.npy')
URI_treino = np.load('../../Corpora/PetroNER-LinkedEntity/treino - URI.npy', allow_pickle=True)

#Valid
#text_valid = np.load('../../Corpora/PetroNER-LinkedEntity/valid - sentences.npy')
entities_valid = np.load('../../Corpora/PetroNER-LinkedEntity/valid - entities.npy')
classes_valid = np.load('../../Corpora/PetroNER-LinkedEntity/valid - classes.npy')
URI_valid = np.load('../../Corpora/PetroNER-LinkedEntity/valid - URI.npy', allow_pickle=True)

#Teste
#text_teste = np.load('../../Corpora/PetroNER-LinkedEntity/teste - sentences.npy')
entities_teste = np.load('../../Corpora/PetroNER-LinkedEntity/teste - entities.npy')
classes_teste = np.load('../../Corpora/PetroNER-LinkedEntity/teste - classes.npy')
URI_teste = np.load('../../Corpora/PetroNER-LinkedEntity/teste - URI.npy', allow_pickle=True)

Carregando arquivo com relações

In [3]:
relations_treino = pd.read_csv('../../Corpora/PetroRE/df_relation_treino.csv')
relations_valid = pd.read_csv('../../Corpora/PetroRE/df_relation_valid.csv')
relations_teste = pd.read_csv('../../Corpora/PetroRE/df_relation_teste.csv')

### Verificando se tem URI com duas classes diferentes

In [4]:
#Coletando tuplas com clases e URIs
entity_tupla = []

for n in range(len(URI_PetroNER)):
    entity_tupla.append((URI_PetroNER[n],classes_PetroNER[n]))

#Set de tuplas
set_tupla = set(entity_tupla)

# Excluido as URIs None
ent = []
for tup in set_tupla:
    #print(tup[0])
    if tup[0] != None:
        ent.append(tup[0])
        
#Ordenando as URIs e imprimindo quando tem uma URI repetida
classes_duplicadas = []
ent.sort()
ant = ''
for i in ent:
    if i == ant:
        print(i)
        classes_duplicadas.append(i)
    ant = i

#TEFR_CD_TIPO_EST_FISICA_ROCHA_059


### Povoamento a ontologia com as entidades e relações

função para receber os datasets e povoar as ontologias

In [5]:
def povoando_ontologia(URI_dataset, classes_dataset, relations_dataset, populated_onto_file_name):
    
    #Carregando ontologias (não povoada)
    world = World()
    onto = world.get_ontology("../OntoGeoLogicaEntidadesNomeadas.owl").load()
    #onto.load()
    
    # Função que recebe as listas com todas URIs e Classes, 
    # e retorna apenas um set das URI dos elementos da classe indicada em 'string_classe', 
    # com exceção da URI da própria classe (indicado em URI_classe)

    def lista_por_classe(URI_dataset, classes_dataset, string_classe, URI_classe):

        lista_classe = []

        #Iterando por todos os elementos
        for n in range(len(URI_dataset)):
            #Verificando qual a classe
            if ((classes_dataset[n] == string_classe) and 
                (URI_dataset[n]!= URI_classe) and 
                (URI_dataset[n]!= None) and
                (URI_dataset[n]not in classes_duplicadas)):
                lista_classe.append(URI_dataset[n])


        # Reduzindo a lista para apenas elementos únicos
        lista_classe = set(lista_classe)

        return lista_classe
    
    #separando as listas com classes e instâncias
    BACIA_list = lista_por_classe(URI_dataset, classes_dataset, 'BACIA', '#basin')
    CAMPO_list = lista_por_classe(URI_dataset, classes_dataset, 'CAMPO', '#field')
    ESTRUTURA_FÍSICA_list = lista_por_classe(URI_dataset, classes_dataset, 'ESTRUTURA_FÍSICA', '#geological_structure')
    EVENTO_PETRO_list = lista_por_classe(URI_dataset, classes_dataset, 'EVENTO_PETRO', '#petroleum_system_events')
    FLUIDO_list = lista_por_classe(URI_dataset, classes_dataset, 'FLUIDO', '#anthropogenic_fluid')
    FLUIDODATERRA_i_list = lista_por_classe(URI_dataset, classes_dataset, 'FLUIDODATERRA_i', '#o&g_earth_fluid')
    FLUIDODATERRA_o_list = lista_por_classe(URI_dataset, classes_dataset, 'FLUIDODATERRA_o', '#earth_fluid')
    NÃOCONSOLID_list = lista_por_classe(URI_dataset, classes_dataset, 'NÃOCONSOLID', '#unconsolidated_earth_material')
    POÇO_list = lista_por_classe(URI_dataset, classes_dataset, 'POÇO', '#well')
    ROCHA_list = lista_por_classe(URI_dataset, classes_dataset, 'ROCHA', '#rock')
    TEXTURA_list = lista_por_classe(URI_dataset, classes_dataset, 'TEXTURA', '#textura')
    UNIDADE_CRONO_list = lista_por_classe(URI_dataset, classes_dataset, 'UNIDADE_CRONO', '#geological_age')
    UNIDADE_LITO_list = lista_por_classe(URI_dataset, classes_dataset, 'UNIDADE_LITO', '#lithostratigraphic_unit')
    ELEMENTO_PETRO_list = lista_por_classe(URI_dataset, classes_dataset, 'ELEMENTO_PETRO', '#petroleum_system_elements')
 
    # Povoando as subclasses
    #Função que recebe a classe da ontologia e a lista de sublasses para povoar
    def povoando_subclasses(onto, classe, lista_classe):

        for new_class in lista_classe:
            with onto:
                types.new_class(new_class[1:], (onto[classe],))
        return
    
    povoando_subclasses(onto, 'rock', ROCHA_list)
    povoando_subclasses(onto, 'unconsolidated_earth_material', NÃOCONSOLID_list)
    povoando_subclasses(onto, 'o&g_earth_fluid', FLUIDODATERRA_i_list)
    povoando_subclasses(onto, 'earth_fluid', FLUIDODATERRA_o_list)
    povoando_subclasses(onto, 'anthropogenic_fluid', FLUIDO_list)
    povoando_subclasses(onto, 'petroleum_system_events', EVENTO_PETRO_list)
    povoando_subclasses(onto, 'petroleum_system_elements', ELEMENTO_PETRO_list)
    
    # Povoando as instâcias
    
    for basin in BACIA_list:
        onto['basin'](basin[1:])

    for field in CAMPO_list:
        onto['field'](field[1:])

    for structure in ESTRUTURA_FÍSICA_list:
        onto['geological_structure'](structure[1:])

    for well in POÇO_list:
        onto['well'](well[1:])

    for textura in TEXTURA_list:
        onto['textura'](textura[1:])

    for lithostratigraphic_unit in UNIDADE_LITO_list:
        onto['lithostratigraphic_unit'](lithostratigraphic_unit[1:])
        
    #Trantando o caso específico da Unidade Cronoestratigráfica.  
    #Para cada URI anotada no corpus, povoamos na ontologia um subclasse de onto.geological_age, 
    #terminando com '_quality' e uma instância de onto.geological_time_interval com a URI anotada no corpus.
    
    for crono in UNIDADE_CRONO_list:
        with onto:
            #povoando as classes geological age e geological time interval
            types.new_class(crono[1:] + '_quality', (onto['geological_age'],))
            onto['geological_time_interval'](crono[1:])

            
    # Povoando a ontologia com as relações
    # Colentando as triplas (URI_1, Relation, URI_2)
    triplas = list(relations_dataset.apply(lambda x: (x['URI_1'], x['Relation'], x['URI_2']), axis=1))
    #print('Número de triplas:', len(triplas))
    triplas = set(triplas)
    #print('Número de triplas únicas:', len(triplas))
    
    for tripla in triplas:
    
        # usamos o try porque ainda estamos usando um conjunto de relações sem split (treino-valid-teste) 
        try:
            if ((tripla[0] not in classes_duplicadas) and 
                (tripla[2] not in classes_duplicadas)):

                if tripla[1] == 'constituted_by':
                    onto[tripla[0][1:]].constituted_by.append(onto[tripla[2][1:]])

                if tripla[1] == 'crosses':
                    onto[tripla[0][1:]].crosses.append(onto[tripla[2][1:]])

                if tripla[1] == 'has_age':
                    # Criando a instância "idade geológica da unidade lito"
                    onto[tripla[2][1:] + '_quality'](tripla[2][1:] + '_quality_' + tripla[0][1:])

                    # Criando a relação entra a unidade lito e instância "idade geológica da unidade lito"
                    onto[tripla[0][1:]].has_age.append(onto[tripla[2][1:] + '_quality_' + tripla[0][1:]])

                    # Adicionando a relação partipates_in entre a qualidade Geological age e o Geological Time interval

                    onto[tripla[2][1:] + '_quality_' + tripla[0][1:]].participates_in.append(onto[tripla[2][1:]])

                if tripla[1] == 'located_in':
                     onto[tripla[0][1:]].located_in.append(onto[tripla[2][1:]])
                        
                if tripla[1] == 'part_of':
                     onto[tripla[0][1:]].part_of.append(onto[tripla[2][1:]])
                        
                if tripla[1] == 'carrier_of':
                     onto[tripla[0][1:]].carrier_of.append(onto[tripla[2][1:]])
                        
                if tripla[1] == 'generated_by':
                     onto[tripla[0][1:]].generated_by.append(onto[tripla[2][1:]])

                if tripla[1] == 'has_temporal_relation':
                    onto[tripla[0][1:]].has_temporal_relation.append(onto[tripla[2][1:]])

        except:
            pass
            
    onto.save(file = populated_onto_file_name)
    
    def n_instancias(cla):

        lista =  list(world.sparql("""
                PREFIX onto: <""" + str(onto.base_iri) + """>
                SELECT ?x
                {?x a onto:""" + cla + """. }
                """))

        return (len(lista))
    
    ### Imprimindo algumas Estatísticas
    print('-----------------------------')
    print('Estatísticas de ',  populated_onto_file_name)
    print('\n ')
    print('Total individuals: ', len(list(onto.individuals())))
    print('Bacias: ', n_instancias('basin'))
    print('Campo: ', n_instancias('field'))
    print('Estrutura Física: ',  n_instancias('geological_structure'))
    print('Poço: ',  n_instancias('well'))
    print('Textura: ',  n_instancias('textura')) 
    print('Unidade Lito: ',  n_instancias('lithostratigraphic_unit')) 
    print('Intervalo de tempo Geológico: ',  n_instancias('geological_time_interval'))
    #print('Idade Geologica: ', len(onto.search(type = onto.geological_age)))
    print('-----------------------------')
    print('Total classes: ', len(list(onto.classes())))
    print('Fluido antropogênico: ', len(list(onto['anthropogenic_fluid'].subclasses())))
    print('Fluido da terra O&G: ', len(list(onto['o&g_earth_fluid'].subclasses())))
    print('Fluido da terra: ', len(list(onto['earth_fluid'].subclasses())))
    print('Material não consolidado: ', len(list(onto['unconsolidated_earth_material'].subclasses())))
    print('Rocha: ', len(list(onto['rock'].subclasses())))
    print('Idade Geologica: ', len(list(onto['geological_age'].subclasses())))
    print('Eventos do Sistema Petrolífero: ', len(list(onto['petroleum_system_events'].subclasses())))
    print('Elementos do Sistema Petrolífero: ', len(list(onto['petroleum_system_elements'].subclasses())))
    print('-----------------------------')
    print('constituted_by : ', len(list(onto.constituted_by.get_relations())))
    print('crosses : ', len(list(onto.crosses.get_relations())))
    print('has_age : ', len(list(onto.has_age.get_relations())))
    print('participates_in : ', len(list(onto.participates_in.get_relations())))
    print('located_in : ', len(list(onto.located_in.get_relations())))
    print('has_temporal_relation : ', len(list(onto.has_temporal_relation.get_relations())))
    print('part_of : ', len(list(onto.part_of.get_relations())))
    print('carrier_of : ', len(list(onto.carrier_of.get_relations())))
    print('generated_by : ', len(list(onto.generated_by.get_relations())))
    print('-----------------------------')
    
    #Apagando a ontologia para não afetar as demais chamadas 
    onto.destroy()
    return

In [6]:
#povoando_ontologia(URI_PetroNER, classes_PetroNER, relations_PetroNER, "PetroKGraph_PetroNER")

In [7]:
povoando_ontologia(URI_treino, classes_treino, relations_treino, "PetroKGraph_treino")

-----------------------------
Estatísticas de  PetroKGraph_treino

 
Total individuals:  645
Bacias:  61
Campo:  92
Estrutura Física:  60
Poço:  124
Textura:  18
Unidade Lito:  173
Intervalo de tempo Geológico:  107
-----------------------------
Total classes:  239
Fluido antropogênico:  1
Fluido da terra O&G:  7
Fluido da terra:  2
Material não consolidado:  9
Rocha:  72
Idade Geologica:  107
Eventos do Sistema Petrolífero:  3
Elementos do Sistema Petrolífero:  3
-----------------------------
constituted_by :  153
crosses :  13
has_age :  10
participates_in :  10
located_in :  257
has_temporal_relation :  0
part_of :  27
carrier_of :  0
generated_by :  0
-----------------------------


In [8]:
povoando_ontologia(URI_valid, classes_valid, relations_valid, "PetroKGraph_valid")

-----------------------------
Estatísticas de  PetroKGraph_valid

 
Total individuals:  174
Bacias:  20
Campo:  18
Estrutura Física:  17
Poço:  29
Textura:  4
Unidade Lito:  26
Intervalo de tempo Geológico:  59
-----------------------------
Total classes:  145
Fluido antropogênico:  1
Fluido da terra O&G:  4
Fluido da terra:  2
Material não consolidado:  6
Rocha:  33
Idade Geologica:  59
Eventos do Sistema Petrolífero:  3
Elementos do Sistema Petrolífero:  2
-----------------------------
constituted_by :  11
crosses :  6
has_age :  1
participates_in :  1
located_in :  33
has_temporal_relation :  0
part_of :  4
carrier_of :  0
generated_by :  0
-----------------------------


In [9]:
povoando_ontologia(URI_teste, classes_teste, relations_teste, "PetroKGraph_teste")

-----------------------------
Estatísticas de  PetroKGraph_teste

 
Total individuals:  293
Bacias:  34
Campo:  37
Estrutura Física:  21
Poço:  43
Textura:  2
Unidade Lito:  77
Intervalo de tempo Geológico:  71
-----------------------------
Total classes:  165
Fluido antropogênico:  1
Fluido da terra O&G:  5
Fluido da terra:  2
Material não consolidado:  6
Rocha:  39
Idade Geologica:  71
Eventos do Sistema Petrolífero:  3
Elementos do Sistema Petrolífero:  3
-----------------------------
constituted_by :  61
crosses :  7
has_age :  8
participates_in :  8
located_in :  67
has_temporal_relation :  0
part_of :  20
carrier_of :  0
generated_by :  0
-----------------------------
